In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from utils import *

In [2]:

# Quantization function
def quantize(tensor, bits):
    scale = 2**bits - 1
    tensor = torch.round(tensor * scale) / scale
    return tensor

In [4]:


# CIFAR-10 data loading and transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define the model
model = models.resnet18(num_classes=10)
model = model.cuda()

100%|██████████| 170498071/170498071 [00:13<00:00, 12576881.96it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Training loop with quantization
num_epochs = 100
quantization_bits = 8  # Adjust this for different levels of quantization

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in trainloader:
        inputs, targets = inputs.cuda(), targets.cuda()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Quantize gradients
        for param in model.parameters():
            if param.grad is not None:
                param.grad.data = quantize(param.grad.data, quantization_bits)

        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

print('Finished Training')


Epoch 1, Loss: 2.0566825766087797
Accuracy: 43.26%
Epoch 2, Loss: 1.3908814079011493
Accuracy: 45.72%
Epoch 3, Loss: 1.1548378362375147
Accuracy: 59.18%
Epoch 4, Loss: 1.0108699850414111
Accuracy: 62.91%
Epoch 5, Loss: 0.9292134128873001
Accuracy: 66.69%
Epoch 6, Loss: 0.8675669321928487
Accuracy: 66.67%
Epoch 7, Loss: 0.8133497939390295
Accuracy: 68.87%
Epoch 8, Loss: 0.7678780334684855
Accuracy: 64.19%
Epoch 9, Loss: 0.7445235591562812
Accuracy: 68.5%
Epoch 10, Loss: 0.7155868035295735
Accuracy: 68.95%
Epoch 11, Loss: 0.6955896713544646
Accuracy: 72.15%
Epoch 12, Loss: 0.6694699579187672
Accuracy: 69.5%
Epoch 13, Loss: 0.6590835974954278
Accuracy: 72.91%
Epoch 14, Loss: 0.6455053931764324
Accuracy: 70.81%
Epoch 15, Loss: 0.6308718001293709
Accuracy: 71.79%
Epoch 16, Loss: 0.6126811268841824
Accuracy: 68.82%
Epoch 17, Loss: 0.597563892755362
Accuracy: 72.78%
Epoch 18, Loss: 0.5923787045966634
Accuracy: 72.86%
Epoch 19, Loss: 0.589088560065345
Accuracy: 71.85%
Epoch 20, Loss: 0.5682069

KeyboardInterrupt: 